# Prediction for one composition

(c) Le Losq et al. 2023
see embedded licence file

**This is a simple notebook to use: follow the instructions, and you will get predictions for a composition of interest.**

In [1]:
# Library loading and additional function definition
%matplotlib inline

# dataframes and arrays
import pandas as pd, numpy as np

# for plotting
import matplotlib.pyplot as plt 

# imelt internals
import src.imelt as imelt
import src.utils as utils

import torch

# we will work on CPU there
device=torch.device('cpu')

# we load the model
imelt_model = imelt.load_pretrained_bagged()

# What is the composition of interest ?

Here we reproduce the prediction for albite glass, change for your case.

In [5]:
composition = pd.DataFrame()

# MODIFY THE FOLLOWING
composition["sio2"] = 0.75
composition["al2o3"] = 0.125
composition["na2o"] = 0.125
composition["k2o"] = 0.0
composition["mgo"] = 0.0
composition["cao"] = 0.0

# some internal workings to add descriptors
composition = utils.descriptors(composition).values

# Viscosity prediction

1/ Select the temperature range


In [6]:
T_start = 2000 # MODIFY THIS, lower T limit
T_stop = 3000 # MODIFY THIS, upper T limit

2/ select the equation, among: 
    
    - ag > Adam-Gibbs
    - am > Avramov Milchev
    - myega > MYEGA
    - tvf > VFT equation
    - cg > Free Volume

In [7]:
equation = "ag" # MODIFY THIS

# Some examples of predictions

Basically, the model object, here called imelt_model, has a `predict` method. You can pass many different parameters here to calculate various properties/variables. We show a brief example here, look at the source code of i-Melt to get further ideas!

In [8]:
T_range = np.arange(T_start, T_stop, 1.0)

viscosity = imelt_model.predict(equation,composition*np.ones((len(T_range),39)),T_range.reshape(-1,1))
tg = imelt_model.predict("tg",composition, sampling=True, n_sample=20)
sctg = imelt_model.predict("sctg", composition, sampling=True, n_sample=20)
fragility = imelt_model.predict("fragility", composition, sampling=True, n_sample=20)
density = imelt_model.predict("density_glass", composition, sampling=True, n_sample=20)
ri = imelt_model.predict("sellmeier", composition, [589.0], sampling=True, n_sample=20)

A_TVF = imelt_model.predict("a_tvf", composition, sampling=True, n_sample=20)
B_TVF = imelt_model.predict("b_tvf", composition, sampling=True, n_sample=20)
C_TVF = imelt_model.predict("c_tvf", composition, sampling=True, n_sample=20)

raman = imelt_model.predict("raman_pred", composition, sampling=True, n_sample=20)

print('')
print('Results for composition with mol fractions')
print('{} SiO2, {} Al2O3, {} Na2O, {} K2O, {} MgO, {} CaO'.format(composition[0,0],composition[0,1],composition[0,2],
                                                                 composition[0,3],composition[0,4],composition[0,5]))
print('')
print('Tg is of {:.0f} +/- {:.0f}'.format(tg.mean(axis=1)[0], tg.std(axis=1)[0]))
print('Sconf at Tg is of {:.1f} +/- {:.1f}'.format(sctg.mean(axis=1)[0], sctg.std(axis=1)[0]))
print('')
print('Fragility is of {:.1f} +/- {:.1f}'.format(fragility.mean(axis=1)[0], fragility.std(axis=1)[0]))
print('Density is of {:.3f} +/- {:.3f}'.format(density.mean(axis=1)[0], density.std(axis=1)[0]))
print('Refractive Index at 598 nm is of {:.3f} +/- {:.3f}'.format(ri.mean(axis=1)[0], ri.std(axis=1)[0]))
print('')
print('Parameter A for VFT is of {:.2f}'.format(A_TVF.mean(axis=1)[0]))
print('Parameter B for VFT is of {:.1f}'.format(B_TVF.mean(axis=1)[0]))
print('Parameter C for VFT is of {:.1f}'.format(C_TVF.mean(axis=1)[0]))

# Plotting the Raman spectra
x_raman_shift = np.arange(400.,1250.,1.0)
plt.figure()
plt.plot(x_raman_shift,raman.mean(axis=2).ravel())
plt.xlabel("Raman shift, cm-1")
plt.ylabel("Intensity, a.u.")
plt.show()

NameError: name 'neuralmodel' is not defined